In [1]:
from nemo_text_processing.text_normalization.sv.taggers.cardinal import CardinalFst as tnCard
from nemo_text_processing.inverse_text_normalization.sv.taggers.cardinal import CardinalFst as Card
from nemo_text_processing.text_normalization.sv.taggers.decimal import DecimalFst as tnDec
from nemo_text_processing.inverse_text_normalization.sv.taggers.decimal import DecimalFst as Dec

In [2]:
tn_card = tnCard(False)
card = Card(tn_card)
tn_dec = tnDec(tn_card, False)
dec = Dec(card, tn_dec)

In [3]:
("en miljon" @ dec.fst).string()

'decimal { integer_part: "1" quantity: "miljon" preserve_order: true }'

In [4]:
("ett miljon" @ dec.fst).string()

'decimal { integer_part: "1" quantity: "miljon" preserve_order: true }'